In [5]:
from bs4 import BeautifulSoup as bs
import requests
import re
import pandas as pd

In [6]:
URL = "https://dantri.com.vn/su-kien.htm"

In [7]:

def get_all_post():
    r = requests.get(URL)
    news_urls = []
    soup = bs(r.text,"lxml")
    for item in soup.select("h3.news-item__title>a"):
        url = item['href']
        news_urls.append(url)
    return news_urls

def get_news_detail(url):
    r = requests.get(url)
    soup = bs(r.text,"lxml")
    title = soup.select_one("h1").text.strip()
    description = soup.find("meta",attrs = {"name":"description"}).get("content", "").strip()
    script_date = re.sub(r'\n+',' ',str(soup.select_one("script")))
    publish_time = re.sub(r'.+"published_date":"([^"]+)?".+',r'\1',script_date).strip()
    return title, description, publish_time
title = []
description = []
publish_time = []
link = []
df = pd.DataFrame()
for url in get_all_post():
    full_url = url
    if full_url.startswith('/'):
        full_url = 'https://dantri.com.vn'+ url
    t, d, p = get_news_detail(full_url)
    if t:
        title.append(t)
        description.append(d)
        publish_time.append(p)
        link.append(url)
df["Title"] = title
df["Description"] = description
df["Publish_Time"] = publish_time
df["Link"] = link
df.to_csv('articles.csv')
    